In [1]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [14]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZjM4NzdlOGItYmZiYy00ZGIyLThiZDMtM2VhMTZiMDJlNGQ0IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI0QnFWdThuYzZyNXAzZzdyY1hiVmRIZllkV1RzVkdyV29vR2hib1dtRUt3UyJdLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [15]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNmNhODgxNDUtOGZkNi00ZmZmLWEzYmQtYjM0MDRhZjBhMmFjIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyI4VmJiOVNBa3FxV2VURnhtN29VUlgzUk1hNUp0cVIzWGM5ekZ2Z1E5OXdoQSJdfQ==


In [4]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '8f0f158b-c9fe-4274-b7d1-a5fcadfb9709',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['74MPyGb42fYYSuuCREkkh4k4yjTxD6oNYHDBf3ssd3F8'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [5]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2hZiozbLxfXyHRB6hg6ZH25iqTprb4rZtXkjJbzzHVMud9BkubmdqWgzgYKRWph7qdRehFwKwarAZqCEZH9xFJ7b

Public key: 
 JDhqfJhhn6aZxTnW192VQYNwJDBHrGJx7kWYH8XLiuvR


**Sending a connection request to acapy**

In [6]:
   
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [7]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [8]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:AhEexLqttHShTSfhfTHTqT#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:AhEexLqttHShTSfhfTHTqT',
             'publicKey': [{'controller': 'did:sov:AhEexLqttHShTSfhfTHTqT',
                            'id': 'did:sov:AhEexLqttHShTSfhfTHTqT#1',
                            'publicKeyBase58': '6HSNTkgQqq6q25SujuUkF6rnvusHbBKHDT4HkMiHqEdK',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:AhEexLqttHShTSfhfTHTqT;indy',
                          'priority': 0,
                          'recipientKeys': ['6HSNTkgQqq6q25SujuUkF6rnvusHbBKHDT4HkMiHqEdK'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'T\xff{\xe1\xce2\x88\x18\x99\xe4I\xa7`\xdb\x92\x8b\xba\x8

### FEATURE DISCOVERY

In [9]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': 'ba82a8c4-ab9a-422e-8576-15fdfbe95111',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'https://didcomm.org/credential-issuance/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-holder/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-credential-definitions/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/records'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
  

In [10]:
message = util.buildMessage(
    'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create',
    label="agent2",
    alias="agent2",
    auto_accept=True 
)
print(message)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': '31e7a4a6-74a9-44ba-9ef9-691b076fe6dd', '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create', '~transport': {'return_route': 'all'}, 'label': 'agent2', 'alias': 'agent2', 'auto_accept': True}
{'@id': 'fda3b581-6e0c-4ad9-8efe-bd73c192df62',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/invitation',
 'alias': 'agent2',
 'auto_accept': True,
 'created_date': '2020-05-25 11:50:25.087020Z',
 'id': '254ac3d7-5a0f-4502-a491-82decb5b8178',
 'invitation_url': 'http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWY0ODc1MzEtNjNiOC00YzAyLWEwMmUtNDllODgyY2E2MzVkIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJFNUt6bU1HWkdkaXFKOGJ0SnE5VFA1VnVpQzQ3NXloaUNLZk1pNlNQOUIzZyJdLCAibGFiZWwiOiAiYWdlbnQyIn0=',
 'label': 'agent2',
 'multi_use': False,
 'raw_repr': {'connection'

In [11]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/receive-invitation",
    accept='auto',
    invitation=response['raw_repr']['invitation'],
)
responsee = util.sendMessage(message, connections[1])
pp.pprint(response)


{'@id': 'fda3b581-6e0c-4ad9-8efe-bd73c192df62',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/invitation',
 'alias': 'agent2',
 'auto_accept': True,
 'created_date': '2020-05-25 11:50:25.087020Z',
 'id': '254ac3d7-5a0f-4502-a491-82decb5b8178',
 'invitation_url': 'http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWY0ODc1MzEtNjNiOC00YzAyLWEwMmUtNDllODgyY2E2MzVkIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJFNUt6bU1HWkdkaXFKOGJ0SnE5VFA1VnVpQzQ3NXloaUNLZk1pNlNQOUIzZyJdLCAibGFiZWwiOiAiYWdlbnQyIn0=',
 'label': 'agent2',
 'multi_use': False,
 'raw_repr': {'connection': {'accept': 'auto',
                             'alias': 'agent2',
                             'connection_id': '254ac3d7-5a0f-4502-a491-82decb5b8178',
                             'created_at': '2020-05-25 11:50:25.087020Z',
                    

In [17]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/connection-get-list",
)
responseee = util.sendMessage(message, connections[0])
pp.pprint(responseee)

{'@id': '82831db6-4456-42c8-84da-a4f262201faa',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/connection-list',
 'results': [{'accept': 'auto',
              'connection_id': '494e3454-e450-4621-8af3-b43a83dd7295',
              'created_at': '2020-05-25 11:50:24.320627Z',
              'initiator': 'self',
              'invitation_key': '4Sybxnii2djC3AmZmSUmjuh8DadV7DRy3MvCR8mtX4ue',
              'invitation_mode': 'once',
              'my_did': 'AhEexLqttHShTSfhfTHTqT',
              'routing_state': 'none',
              'state': 'active',
              'their_did': 'YbNUr7Coh2SYfW9kSWRK9B',
              'their_label': 'Plugin',
              'their_role': 'admin',
              'updated_at': '2020-05-25 11:50:24.972160Z'},
             {'accept': 'auto',
              'alias': 'agent2',
              'connection_id': '254ac3d7-5a0f-4502-a491-82decb5b8178',
              'created_at': '2020-05-25 11:50:25.087020Z',
              'initiator': 'self',
      